## Voy a testear la versión del MODL para OPT data

In [1]:
!git config --global user.email 'obandomarcos96@gmail.com'
!git config --global user.password 'Quovadis96'

Me posiciono en el directorio del GIT

In [4]:
import os
os.chdir('/content/drive/MyDrive/Balseiro/Maestría/Implementación/DeepOPT')

In [5]:
!git add Tests/HDF5_dataset_test.ipynb  Tests/MODL_training_test.ipynb Tests/OPT_MODL_test.ipynb;git commit -m ''

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)

	modified:   Tests/HDF5_dataset_test.ipynb
	modified:   Tests/MoDL_training_test.ipynb
	modified:   modl (modified content, untracked content)

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Datasets/
	DatasetsOPTdatasets.hdf5
	OPTmodl/__pycache__/
	Tests/OPT_MODL_test.ipynb
	Utilities/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")
